In [ ]:
from pylab import *
%matplotlib inline

from skimage import *
from skimage.filters import *
from scipy import ndimage
import numpy as np
import scipy

# Prvi zadatak

In [ ]:
image = img_as_float(imread('girl_ht.tif'))

figure(dpi=200,figsize=(16,8));
imshow(image,cmap='gray');
axis('off');

In [ ]:
# Prvo je urađeno usrednjavanje Gausovim filtrom. Ovim fitriranjem se nisu izgubili bitni detalji, ali prelazi između belih i crnih tačkica nisu više nagli.
res1 = gaussian(image,sigma=5);
RES1 = fftshift(fft2(res1))

figure(dpi=200,figsize=(16,8));
imshow(res1,cmap='gray');
axis('off');

In [ ]:
def cnotch(filt_type, notch, M, N, C, r, n=1):
    filter_mask = zeros([M,N])
    y = np.arange(0,M) - M/2 + 0.5;
    x = np.arange(0,N) - N/2 + 0.5;

    X,Y = meshgrid(x, y)

    for i in range(0, len(C)):
        C_current = C[i]

        C_complement = zeros(2)
        C_complement[0] = -C_current[0]
        C_complement[1] = -C_current[1]

        y0 = y - C_current[0] + (M-1)/2;
        x0 = x - C_current[1] + (N-1)/2;

        X0, Y0 = meshgrid(x0,y0)
        D0 = np.sqrt(np.square(X0) + np.square(Y0))
            
        y0c = y - C_complement[0] - (M-1)/2;
        x0c = x - C_complement[1] - (N-1)/2;

        X0c, Y0c = meshgrid(x0c,y0c)
        D0c = np.sqrt(np.square(X0c) + np.square(Y0c))

        filter_mask += exp(-np.square(D0)/(2*np.square(r)))
        filter_mask += exp(-np.square(D0c)/(2*np.square(r)))

    return 1 - filter_mask

In [ ]:
#U ovom koraku su pomoću notch filtra iz frekvencijskog domena uklonjeni pikovi koji se ne nalaze u blizini DC komponente.
C = []
C.append([673,1090])
C.append([600,1090])
C.append([673,1240])
C.append([673,1392])

[M,N] = shape(image)
H = cnotch('gaussian', 'reject', M, N, C, 5)

RES2 = H*RES1
res2 = abs(ifft2(ifftshift(RES2)));

figure(dpi=200,figsize=(16,8));
imshow(res2,cmap='gray');
axis('off');

In [ ]:
# U ovom koraku su izfiltrirani preostale crne i bele tačke.
res3 = gaussian(res2,sigma=6);

figure(dpi=200,figsize=(16,8));
imshow(res3,cmap='gray');
axis('off');

# Drugi zadatak

Pri izradi ovog zadatka su posmatrani histogrami varijansi lokalnog susedstva ulazne slike za različite vrednosti radiusa lokalnog susedstva. Kao predpostavka varijanse šuma prisutnog u slici je uzeta ona vrednost koja ima najveću vrednost u jednom pojedinačnom histogramu gledano u odnosu na sve histograme. To je vrednost 0,0073 koja se najčešće pojavljuje kada je radius lokalonog susedstva 7. Potom je za ove vrednosti lokoalnog susedstva i varijanse primenjeno adaptivno filtriranje koje pri filtriranju osim ovih vrednosti uzuima u obzir varijansu lokalnog susedtva svakog pojedinačnog piksela.

In [ ]:
# Računanje vrednosti varijanse lokalnog susedstva svakog piksela, pored toga ova funkcija kao prvi argument vraća i vraća i srednju vrednost lokalnog susedstva svakog piksela
def loc_var(image,radius):
    image2 = np.square(image);
    mask = np.ones((2*radius+1,2*radius+1))/((2*radius+1)**2)

    e_im = ndimage.correlate(image, mask)
    e_im2 = ndimage.correlate(image2, mask)
    return e_im, (e_im2 - e_im**2);

In [ ]:
# Filtriranje primenom adaptivnog filtriranja
def denoise(image,r):
    e_im, var = loc_var(image,r)
    a, b = np.histogram(var.flatten(), bins=3000)
    noise_var = b[np.where(max(a)==a)];

    return image + noise_var/var*(e_im-image);

In [ ]:
image = img_as_float(imread('lena_noise.tif'));

fig, axes = subplots(nrows=4,ncols=5,figsize=(25,20),dpi =100)
ax = axes.ravel()
tight_layout()

max_pix = 0
max_radius = 0
max_var = 0

# Prolazak kroz sve histograma i traženje najčešće javljanje varijanse
for i in range(1,21):
    _,var = loc_var(image,i)
    a, b = np.histogram(var.flatten(), bins=3000)
    noise_var = b[np.where(max(a)==a)];
    if(max(a)>max_pix):
        max_pix = max(a);
        max_radius = i;
        max_var = b[np.where(max(a)==a)];
    ax[i-1].plot(b[:shape(b)[0]-1],a)
    ax[i-1].set_title("Radius = "+str(i))
print("Odabrani radius je: "+str(max_radius))
print("Dobije varijansa je: "+str(max_var[0]))

In [ ]:
filt_image = denoise(image,max_radius)

fig, axes = subplots(nrows=1,ncols=2,dpi =200,figsize=(16,8))
ax = axes.ravel()
tight_layout()

ax[0].imshow(image,cmap='gray');
ax[0].axis('off');

ax[1].imshow(filt_image,cmap='gray')
ax[1].axis('off');
# Adaptivno filtriranje je dobro uklonilo šum na uniformnim delovima, ali ga nije dobro uklonilo u blizini ivica. To je posledica toga što je varijansa lokalnog susedstva jako velika u blizini ivica pa je sam filtar bio slab. U slučaju da je primenjeno klasično usrednjavanje ove ivece biti bile zamućene.

# Treći zadatak

Pri izradi ovog zadatka je korišćen metod za procenu varijanse Gausovog šuma u zamućenoj slici predložen u radu P Robinson, Y Roodt, A Nel "Gaussian blur identification using scale-space theory". Ovaj rad navodi da ukoliko na već zamućenu sliku primenimo Gasusov filter različitih vrednosti standardne devijacije i računamo razliku između slike pre i posle dodatnog zamućenja, prvi izvod ove razlike će biti najveći baš za standardnu devijaciju Gausovog zamućenja koje je prisutno u ulaznoj slici. Ovo je dokazano primenom ove metoda na sliku Gausovog filtra određene standardne devijacije. Pošto su operacije oduzimanja i konvolucije linearne one se dokaz može primeniti na početni problem. 

In [ ]:
im = img_as_float(imread('road_blur.png'))

fig, axes = subplots(nrows=2,ncols=2,figsize=(16,16), dpi=100);
ax = axes.ravel();

#Zbog složenosti operacije prvo se radi grublja procena pa sve preciznija porcena.
res = []
for i in range(0,10):
    im1 = np.zeros_like(im);
    im1[:,:,0] = gaussian(im[:,:,0],sigma=i)#Dodoavanje dodatnog zamućenja na svaku komponentu
    im1[:,:,1] = gaussian(im[:,:,1],sigma=i)
    im1[:,:,2] = gaussian(im[:,:,2],sigma=i)
    im1[:,:,3] = im[:,:,3]
    res.append(sum(abs(im-im1)));#Traženje razlike između ulazne slike i dodatno zamućene slike
der = np.array(res[1:shape(res)[0]])-np.array(res[0:(shape(res)[0]-1)]);# Prvi izvod zavisnosti apsolutno razlike između ulazne slike i dodatno zamućene slike od standardne devijacije dodatnog Gausovog filtra
ax[0].plot(np.linspace(1,9,9),der);
print("Varijansa posle 1. iteracije je: "+str(np.where(der==max(der))[0][0]+1))# Određivanje vrednosti za koju je prvi izvod najveći

res = []
for i in range(100,300):
    im1 = np.zeros_like(im);
    im1[:,:,0] = gaussian(im[:,:,0],sigma=i*0.01)
    im1[:,:,1] = gaussian(im[:,:,1],sigma=i*0.01)
    im1[:,:,2] = gaussian(im[:,:,2],sigma=i*0.01)
    im1[:,:,3] = im[:,:,3]
    res.append(sum(abs(im-im1)));
der = np.array(res[1:shape(res)[0]])-np.array(res[0:(shape(res)[0]-1)]);
ax[1].plot(np.linspace(1.01,2.99,199),der);
print("Varijansa posle 2. iteracije je: "+str(np.where(der==max(der))[0][0]*0.01))

res = []
for i in range(1610,1630):
    im1 = np.zeros_like(im);
    im1[:,:,0] = gaussian(im[:,:,0],sigma=i*0.001)
    im1[:,:,1] = gaussian(im[:,:,1],sigma=i*0.001)
    im1[:,:,2] = gaussian(im[:,:,2],sigma=i*0.001)
    im1[:,:,3] = im[:,:,3]
    res.append(sum(abs(im-im1)));
der = np.array(res[1:shape(res)[0]])-np.array(res[0:(shape(res)[0]-1)]);
ax[2].plot(np.linspace(1.611,1.629,19),der);
print("Varijansa posle 3. iteracije je: "+str((1611+np.where(der==max(der))[0][0])*0.001))

res = []
for i in range(16245,16255):
    im1 = np.zeros_like(im);
    im1[:,:,0] = gaussian(im[:,:,0],sigma=i*0.0001)
    im1[:,:,1] = gaussian(im[:,:,1],sigma=i*0.0001)
    im1[:,:,2] = gaussian(im[:,:,2],sigma=i*0.0001)
    im1[:,:,3] = im[:,:,3]
    res.append(sum(abs(im-im1)));
der = np.array(res[1:shape(res)[0]])-np.array(res[0:(shape(res)[0]-1)]);
ax[3].plot(np.linspace(1.6246,1.6254,9),der);
print("Varijansa posle 3. iteracije je: "+str((16246+np.where(der==max(der))[0][0])*0.0001))
sigma = (16246+np.where(der==max(der))[0][0])*0.0001;# Dobijena vrednost standarde vrednosti

In [ ]:
#Funkcija koja primenom Wiener-ovog filtra filtrira sliku
def deblur(sigma,im):
    
    yg = list(range(-159,159))
    xg = list(range(-309,309))

    Xg,Yg = meshgrid(xg, yg)

    h_gaus = (1/(2*pi*(sigma**2)))*exp(-(np.square(Xg) + np.square(Yg))/(2*(sigma**2)))
    h_gaus /= sum(h_gaus)
    H_motion = abs(fftshift(fft2(h_gaus)))# Estimirano zanumćenje skike

    # Parametri Wiener-ovog filtra
    k = 1e-4
    W = (abs(H_motion)**2)/(abs(H_motion)**2+k)

    res1 = np.zeros_like(im)
    res1[:,:,3] = im[:,:,3]

    # Primena Wienerow filtra na svaku komponentu slike
    IM = fftshift(fft2(im[:,:,0]))
    res1[:,:,0] = real(ifft2(ifftshift((IM/H_motion)*W)))
    IM =fftshift(fft2(im[:,:,1]))
    res1[:,:,1] = real(ifft2(ifftshift((IM/H_motion)*W)))
    IM = fftshift(fft2(im[:,:,2]))
    res1[:,:,2] = real(ifft2(ifftshift((IM/H_motion)*W)))

    res1[res1>1] = 1;
    res1[res1<0] = 0;
    
    return res1;

In [ ]:
figure(dpi=250,figsize=(16,8))
imshow(im);
axis('off');
figure(dpi=250,figsize=(16,8))
imshow(deblur(sigma,im));
axis('off');
#Dobijeni rezultat je prihvatljiv. Broj tablica je AWCA-510

# Četvrti zadatak

In [ ]:
#1D maska koja se koristi u prostornom filtru 
def gauss_mask1D(radius, std_dev):
    x = np.arange(0,2*radius+1) - radius;
    res_mask = np.exp(-np.square(x)/(2*np.square(std_dev)));
    res_mask /= sum(res_mask);
    return res_mask[:, np.newaxis];

In [ ]:
#2D maska koja se koristi u frekvenijskom filtru
def gauss_mask2D(radius, std_dev):
    x = np.arange(0,2*radius+1) - radius;
    y = np.arange(0,2*radius+1) - radius;
    X,Y = np.meshgrid(x, y);
    G = np.sqrt(np.square(X)+np.square(Y));
    res_mask = np.exp(-np.square(G)/(2*np.square(std_dev)));
    res_mask /= sum(res_mask);
    return res_mask;

In [ ]:
#Prosirivanje slike istim pikselima
def image_extend(image, radius):
    ext_img = np.zeros((shape(image)[0]+2*radius,shape(image)[1]+2*radius));
    ext_img[radius:shape(image)[0]+radius,radius:shape(image)[1]+radius] = image;
    for i in range(0,radius):
        ext_img[i,radius:shape(image)[1]+radius] = image[0,0:shape(image)[1]];
        ext_img[shape(ext_img)[0]-i-1,radius:shape(image)[1]+radius] = image[shape(image)[0]-1,0:shape(image)[1]];
    for i in range(0,radius):
        ext_img[:,i] = ext_img[:,radius];
        ext_img[:,shape(ext_img)[1]-i-1] = ext_img[:,shape(ext_img)[1]-1-radius];
    return ext_img;

In [ ]:
#Filtriranje slike u prostornom domenu
# Implementiranja je separabilnost
# Pokusao sam da koristim i simetricnost, ali ostvaruje losije perormanse
def filter_gauss(image, radius, std_dev):
    mask = gauss_mask1D(radius,std_dev);
    ext_img = image_extend(image,radius);
    res_img = np.zeros((shape(image)[0],shape(image)[1]+2*radius));
    res_img1 = np.zeros_like(image);
    for i in range(0,shape(image)[0]): #Filtriranje po prvoj osi
        res_img[i,:] = sum(mask*ext_img[i:(i+2*radius+1),:],axis=0);
    mask1 = np.transpose(mask);
    for ii in range(0,shape(image)[1]): #Filtriranje po drugoj osi
        res_img1[:,ii] = sum(mask1*res_img[:,ii:(ii+2*radius+1)],axis=1);
    return res_img1;

In [ ]:
#Filtririanje slike u frekvencijskom domenu
def filter_gauss_freq(image, radius, std_dev):
    ext_img = image_extend(image, radius);
    ext_mask = np.zeros_like(ext_img);
    ext_mask[0:2*radius+1,0:2*radius+1] = gauss_mask2D(radius, std_dev);
    # Pošto je fft jedan od divide-and-conquer algoritama on radi brže ako mu je dat na ulazu tačno određen broj podataka.
    # Inače taj broj je stepen broja 2. U ovom slučaju to nije tako. Sledeća funkcija određuje sledeći broj podataka na ulazu s kojim će algoritam raditi brže.
    siz = scipy.fft.next_fast_len(shape(ext_img)[0]);
    EXT_IMG = scipy.fft.fft2(ext_img,s = (siz,siz));
    EXT_MASK = scipy.fft.fft2(ext_mask,s = (siz,siz));
    EXT_RES_IMG = EXT_IMG*EXT_MASK;
    ext_res_img = abs(scipy.fft.ifft2(EXT_RES_IMG));
    res_img = np.zeros_like(image)
    res_img = ext_res_img[2*radius:shape(image)[0]+2*radius,2*radius:shape(image)[1]+2*radius];
    return res_img;

In [ ]:
image = img_as_float(imread('lena.tif'));

time_spat = []
time_freq = []
time_builtin = []
for i in range(1,51):
    start = time.time()
    tmp1 = filter_gauss(image,i,i/3.0);
    end = time.time()
    time_spat.append(end-start)

    start = time.time()
    tmp2 = filter_gauss_freq(image,i,i/3.0);
    end = time.time()
    time_freq.append(end-start)

    start = time.time()
    tmp3 = gaussian(image,i/3.0,truncate=3);
    end = time.time()
    time_builtin.append(end-start)
figure(dpi=250,figsize=(16,8))
plot(np.linspace(1,50,50),time_spat);
plot(np.linspace(1,50,50),time_freq);
plot(np.linspace(1,50,50),time_builtin);
legend(('Prostorni domen', 'Frekvencijski domen', 'Ugrađena funkcija'));
# Gausov filtar implementiran u prostornom i frekvencijskom domenu imaju slično vreme izvršavanja za radius od 1 do 4. Za ostale vrednosti radiusa funkcija u frekvencijskom domenu je konstantna,
# dok funkcija u prostornom domenu raste u zavisnosti od radiusa filtra što je i očekivano ponašaje. Ručno implementirane funkcije se sporijej izvršavaju u odnonu na ugrađenu funkciju najverovatnije
# zbog načina na koji su implementirane.